In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install PyMuPDF chromadb sentence-transformers tqdm regex
!pip install pdfplumber
!pip install fuzzywuzzy python-Levenshtein unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 12.7 MB/s eta

In [ ]:
import fitz
import re
import unicodedata
from tqdm import tqdm

PDF_PATH = "so_tay_2.pdf"
OUTPUT_TXT = "so_tay_clean.txt"
CHUNKS_TXT = "chunks.txt"

# ========== 1️⃣ Làm sạch cơ bản ==========
def normalize_text(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)
    text = text.replace("–", "-")
    text = re.sub(r"(?<=\d),(?=\d)", ".", text)
    text = re.sub(r"[^\n a-zA-ZÀ-Ỹà-ỹ0-9.,:;()–\-+•\[\]]", " ", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\s+([,.:;)\]\}])", r"\1", text)
    return text.strip()

# ========== 2️⃣ Ghép bảng thành dòng ngang ==========
def fix_grade_blocks(text: str) -> str:
    """
    Ghép các dòng dạng thang điểm dọc thành 1 dòng ngang liền mạch.
    """
    # Ghép các pattern kiểu “8.5 - 10 A” hoặc “B + 3.5”
    lines = text.splitlines()
    fixed_lines = []
    buffer = ""
    for line in lines:
        line = line.strip()
        if not line:
            if buffer:
                fixed_lines.append(buffer.strip())
                buffer = ""
            continue

        # Nối các dòng có vẻ là bảng điểm
        if re.match(r"^[0-9.,\-\+\sa-zA-Z]+$", line):
            buffer += " " + line
        else:
            if buffer:
                fixed_lines.append(buffer.strip())
                buffer = ""
            fixed_lines.append(line)

    if buffer:
        fixed_lines.append(buffer.strip())

    # Làm gọn lại các dòng bảng
    result = []
    for l in fixed_lines:
        # Ví dụ: "8.5 - 10 A 7.8 - 8.4 B + 7.0 - 7.7 B" → tách bằng regex và nối bằng "; "
        if re.search(r"\d+\.\d+\s*-\s*\d+\.\d+", l):
            parts = re.findall(r"\d+(?:\.\d+)?\s*-\s*\d+(?:\.\d+)?\s*[A-Z\+]*", l)
            if parts:
                result.append("; ".join(p.strip() for p in parts))
            else:
                result.append(l)
        else:
            result.append(l)
    return "\n".join(result)

# ========== 3️⃣ Tách điều / mục ==========
def split_sections(text):
    text = re.sub(r"Điều\s*\n\s*(\d+\.)", r"Điều \1", text, flags=re.IGNORECASE)
    text = re.sub(r"Mục\s*\n\s*(\d+\.)", r"Mục \1", text, flags=re.IGNORECASE)
    sections = re.split(r"(?=(?:\n?Điều\s+\d+\.|\n?Mục\s+\d+\.))", text, flags=re.IGNORECASE)
    sections = [s.strip() for s in sections if len(s.strip()) > 150]
    return sections

# ========== MAIN ==========
if __name__ == "__main__":
    print("🚀 Đang đọc PDF & ghép bảng điểm ngang...")
    doc = fitz.open(PDF_PATH)
    pages = []
    for page in tqdm(doc, desc="📖 Trích xuất"):
        text = page.get_text("text")
        text = normalize_text(text)
        text = fix_grade_blocks(text)
        pages.append(text)
    doc.close()

    full_text = "\n\n".join(pages)
    full_text = re.sub(r"SỔ TAY SINH VIÊN KHÓA\s*\d+", "", full_text)
    full_text = re.sub(r"\[\s*Trang\s*\d+\s*\]", "", full_text)
    full_text = re.sub(r"\n{3,}", "\n\n", full_text)

    with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
        f.write(full_text)
    print(f"✅ Lưu văn bản sạch: {OUTPUT_TXT}")

    chunks = split_sections(full_text)
    with open(CHUNKS_TXT, "w", encoding="utf-8") as f:
        f.write("\n\n".join(chunks))
    print(f"✅ Tổng {len(chunks)} đoạn đã lưu vào {CHUNKS_TXT}")


🚀 Đang đọc PDF & ghép bảng điểm ngang...


📖 Trích xuất: 100%|██████████| 232/232 [00:01<00:00, 182.88it/s]

✅ Lưu văn bản sạch: so_tay_clean.txt
✅ Tổng 197 đoạn đã lưu vào chunks.txt


In [4]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
COLLECTION = "so_tay_hcmue"
EMB_MODEL_NAME = "BAAI/bge-m3"
INPUT_TXT = "chunks.txt"

with open(INPUT_TXT, "r", encoding="utf-8") as f:
    docs = [x.strip() for x in f.read().split("\n\n") if len(x.strip()) > 80]

print(f"📘 Tổng {len(docs)} đoạn nội dung.")

client = chromadb.PersistentClient(path=DB_DIR, settings=Settings(allow_reset=True))
try:
    client.delete_collection(COLLECTION)
except:
    pass
col = client.create_collection(COLLECTION)

model = SentenceTransformer(EMB_MODEL_NAME)
print("🚀 Đang tạo embeddings...")

embs = []
for i in tqdm(range(0, len(docs), 32)):
    batch = docs[i:i+32]
    batch_emb = model.encode(batch, normalize_embeddings=True).tolist()
    embs.extend(batch_emb)

col.add(ids=[str(i) for i in range(len(docs))],
        documents=docs,
        embeddings=embs)

print(f"✅ Đã lưu {len(docs)} đoạn vào collection '{COLLECTION}'.")


KeyboardInterrupt: 

# Tải lại Collection

In [3]:
import chromadb
from chromadb.config import Settings
import os

# === 1️⃣ Cấu hình để Tải lại Collection ===

# Ghi chú: "/content/drive" là thư mục gốc sau khi Mount Google Drive
# Hãy đảm bảo đường dẫn này khớp chính xác với vị trí bạn lưu folder 'vector_store'
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
COLLECTION = "so_tay_hcmue"

# Kiểm tra xem đường dẫn Drive đã được mount chưa và thư mục DB có tồn tại không
if not os.path.isdir("/content/drive"):
    print("⚠️ Lỗi: Google Drive chưa được Mount. Hãy chạy lệnh Mount Drive trước!")
elif not os.path.isdir(DB_DIR):
    print(f"⚠️ Lỗi: Không tìm thấy thư mục DB tại: {DB_DIR}")
    print("Vui lòng kiểm tra lại đường dẫn Drive đã chính xác chưa.")
else:
    # === 2️⃣ Tải lại Collection ===
    try:
        # Khởi tạo client, nó sẽ tự động load dữ liệu từ DB_DIR
        # KHÔNG dùng allow_reset=True vì ta chỉ muốn tải chứ không muốn tạo mới.
        client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())

        # Lấy Collection đã tồn tại (get_collection)
        col = client.get_collection(COLLECTION)

        # 3. Kiểm tra Collection
        print(f"\n✅ Đã tải thành công Collection '{COLLECTION}' từ Drive.")
        print(f"Tổng số tài liệu trong Collection: {col.count()}")

        # Bạn có thể tiếp tục với các truy vấn tìm kiếm (query) ở đây:
        # Ví dụ:
        # results = col.query(
        #     query_texts=["Điều kiện để được xét tốt nghiệp là gì?"],
        #     n_results=3
        # )
        # print(results)

    except ValueError as e:
        print(f"\n❌ Lỗi ChromaDB: {e}")
        print("Lỗi này thường xảy ra nếu Collection chưa tồn tại. Hãy kiểm tra lại tên COLLECTION.")
    except Exception as e:
        print(f"\n❌ Lỗi không xác định: {e}")


✅ Đã tải thành công Collection 'so_tay_hcmue' từ Drive.
Tổng số tài liệu trong Collection: 409


# Tải model

In [6]:
# Đường dẫn nơi bạn muốn lưu mô hình trên Google Drive
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'

# Tạo thư mục nếu nó chưa tồn tại
os.makedirs(DRIVE_MODEL_PATH, exist_ok=True)
print(f"Thư mục lưu mô hình: {DRIVE_MODEL_PATH}")

Thư mục lưu mô hình: /content/drive/MyDrive/chatbotHCMUE/bge-m3-model/


In [8]:
from sentence_transformers import SentenceTransformer
EMB_MODEL_NAME = "BAAI/bge-m3"

# Khởi tạo mô hình (đã tải hoặc đang tải nếu chưa có trong cache)
model = SentenceTransformer(EMB_MODEL_NAME)

# Lưu mô hình vào Drive
# Đây là bước quan trọng nhất
model.save(DRIVE_MODEL_PATH)

print("✅ Đã lưu mô hình thành công vào Google Drive.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

✅ Đã lưu mô hình thành công vào Google Drive.


In [20]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import re, json, pandas as pd
from fuzzywuzzy import fuzz
from unidecode import unidecode
import chromadb.api

# === 0️⃣ Hàm Loại bỏ Dấu Tiếng Việt (Hỗ trợ Fuzzy Matching) ===
def remove_vietnamese_diacritics(text):
    """Sử dụng unidecode để loại bỏ dấu tiếng Việt và chuyển sang chữ thường."""
    return unidecode(text).lower()


# === 1️⃣ Cấu hình ===
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'
COLLECTION = "so_tay_hcmue"
TABLE_JSON = "/content/drive/MyDrive/chatbotHCMUE/so_tay_all_tables_clean.json"


# === 3️⃣ Hàm in đẹp text ===
def pretty(text: str) -> str:
    text = re.sub(r"([;:.\)\]\}])\s*(?=[\+\-•–])", r"\1\n", text)
    text = re.sub(r"\s*([+\-•–])\s+", r"\n\1 ", text)
    text = re.sub(r"\s*(Điều\s+\d+\.)", r"\n\1", text, flags=re.IGNORECASE)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


# === 4️⃣ Hàm tra bảng JSON (Cải tiến Khớp Mờ Chính Xác) ===
def find_table_by_keyword(query: str):

    normalized_query = remove_vietnamese_diacritics(query)

    mapping = {
        # Thêm các key đơn giản để tăng khả năng khớp chính xác
        "4 sang chữ": ["thang_diem_4"],
        "học bổng": ["xep_loai_hoc_bong"],
        "4 sang 10": ["thang_diem_10_4"],
        "thang điểm 10 sang 4": ["thang_diem_10_4"],
        "thang điểm 4 sang chữ": ["thang_diem_4"],
        "xếp loại học lực": ["xep_loai_hoc_luc"],
        "xếp loại học bổng": ["xep_loai_hoc_bong"],
        "yêu cầu học bổng": ["yeu_cau_hoc_bong"],
        "điểm rèn luyện": ["diem_ren_luyen1", "diem_ren_luyen2", "diem_ren_luyen3", "diem_ren_luyen4", "diem_ren_luyen5"],
    }

    BEST_MATCH_SCORE = 90 # ĐIỀU CHỈNH: Tăng ngưỡng tin cậy lên 90 để tránh khớp nhầm
    best_match_key = None
    best_score = 0
    final_result = ""

    # 1. KIỂM TRA KHỚP HOÀN HẢO (CHÍNH XÁC) TRƯỚC
    for map_key in mapping.keys():
        if normalized_query == remove_vietnamese_diacritics(map_key):
            best_match_key = map_key
            best_score = 100
            break

    # 2. KHỚP MỜ (FUZZY MATCHING)
    if not best_match_key:
        for map_key, types in mapping.items():
            normalized_map_key = remove_vietnamese_diacritics(map_key)

            # Sử dụng WRatio và Partial Ratio để có kết quả chính xác nhất
            score_wratio = fuzz.WRatio(normalized_query, normalized_map_key)
            score_partial = fuzz.partial_ratio(normalized_query, normalized_map_key)

            # Ưu tiên điểm cao nhất từ hai phương pháp
            score = max(score_wratio, score_partial)

            if score > best_score and score >= BEST_MATCH_SCORE:
                best_score = score
                best_match_key = map_key

    # 3. Xử lý và THU THẬP TẤT CẢ các bảng liên quan
    if best_match_key:
        types_to_export = mapping[best_match_key]

        final_result += f"### 📊 Kết quả Tra cứu Bảng (Độ khớp: {best_score}%)\n"

        for type_name in types_to_export:
            for t in tables:
                if t["type"] == type_name:
                    df = pd.DataFrame(t["data"])

                    table_title = t.get("title", type_name.replace("_", " ").title())

                    final_result += f"\n#### Bảng: {table_title}\n"
                    final_result += df.to_markdown(index=False)
                    final_result += "\n"

    # 4. Trả về kết quả tổng hợp
    if final_result:
        return final_result
    else:
        return None


# === 5️⃣ Giao diện chat CLI ===
print("=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE (có cả bảng) ===")
print("Nhập 'exit' để thoát.\n")

# === 2️⃣ Nạp dữ liệu ===
# Reset instance tránh lỗi "already exists"
chromadb.api.client.SharedSystemClient._instance = None
client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())
col = client.get_collection(COLLECTION)
model = SentenceTransformer(DRIVE_MODEL_PATH)

try:
    with open(TABLE_JSON, "r", encoding="utf-8") as f:
        tables = json.load(f)
except FileNotFoundError:
    print("⚠️ Không tìm thấy file bảng JSON. Chỉ tra cứu văn bản được.")
    tables = []


while True:
    q = input("📘 Nhập nội dung muốn tra cứu: ").strip()
    if not q:
        continue
    if q.lower() == "exit":
        print("👋 Tạm biệt!")
        break

    # KIỂM TRA LỖI 2: Yêu cầu nhập thêm khi chuỗi quá ngắn
    if len(remove_vietnamese_diacritics(q)) < 5:
        print("⚠️ Vui lòng nhập chi tiết hơn (ví dụ: 'điểm rèn luyện', 'học bổng').")
        print("\n-----------------------------\n")
        continue

    # --- Kiểm tra có bảng phù hợp không ---
    table_result = find_table_by_keyword(q)
    if table_result:
        print("\n🔎 Kết quả tìm thấy (BẢNG):\n")
        print(table_result)
        print("\n-----------------------------\n")
        continue

    # --- Nếu không có bảng, tra văn bản như cũ ---
    q_emb = model.encode(q, normalize_embeddings=True).tolist()
    res = col.query(query_embeddings=[q_emb], n_results=3)

    print("\n🔎 Kết quả tìm thấy (VĂN BẢN):")
    for i, doc in enumerate(res["documents"][0]):
        print(f"\n📄 Đoạn {i+1}:")
        print(pretty(doc))
    print("\n-----------------------------\n")

=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE (có cả bảng) ===
Nhập 'exit' để thoát.

📘 Nhập nội dung muốn tra cứu: chữ sang 4

🔎 Kết quả tìm thấy (BẢNG):

### 📊 Kết quả Tra cứu Bảng (Độ khớp: 95%)

#### Bảng: Thang Diem 4
| Thang điểm chữ   | Thang điểm 4   |
|:-----------------|:---------------|
| A                | 4,0            |
| B+               | 3,5            |
| B                | 3,0            |
| C+               | 2,5            |
| C                | 2,0            |
| D+               | 1,5            |
| D                | 1,0            |
| F+               | 0,5            |
| F                | 0,0            |


-----------------------------

📘 Nhập nội dung muốn tra cứu: 4 sang chữ

🔎 Kết quả tìm thấy (BẢNG):

### 📊 Kết quả Tra cứu Bảng (Độ khớp: 100%)

#### Bảng: Thang Diem 4
| Thang điểm chữ   | Thang điểm 4   |
|:-----------------|:---------------|
| A                | 4,0            |
| B+               | 3,5            |
| B                | 3,0            |
| C+   

KeyboardInterrupt: Interrupted by user